# Klasyfikacja zbioru *CIFAR-10* przy pomocy konwolucyjnych sieci neuronowych
## Autor: Jadwiga Słowik

In [0]:
from matplotlib import pyplot
from scipy.misc import toimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras
import tensorflow as tf
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


# Wczytanie zbioru danych (uczący + testowy)

In [0]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
num_classes=10

170500096/170498071 [==============================] - 18s 0us/step


# Podział zbioru uczącego na treningowy i walidacyjny (w stosunku 9:1)

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_val = X_train[0:5000,:]
y_val = y_train[0:5000,:]
X_train = X_train[5000:50000,:]
y_train = y_train[5000:50000,:]

# Zapisywanie wyników

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json
def save_file(file_name, content):
  with open("/content/gdrive/My Drive/{0}".format(file_name), "w") as f:
    f.write(json.dumps(content))

# Budowanie modelu i testowanie

In [0]:
def create_base_model(
    input_shape,
    number_of_classes,
    conv_num,
    pooling_iter_num,
    fc_num,
    optimizer,
    dropout_fc,
    dropout_conv,
    batch_normalization):
  model = Sequential()
  model.add(Conv2D(128, (3, 3), padding="same", input_shape=input_shape))
  model.add(Activation("relu"))
  
  for j in range(pooling_iter_num):
    for i in range(conv_num):
      model.add(Conv2D(128, (2, 2), padding="same", activation="relu"))
      if dropout_conv != None:
        model.add(Dropout(dropout_conv))
      if batch_normalization != None:
        model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2,2), strides=2))
  
  model.add(Flatten())
  for i in range(fc_num):
    model.add(Dense(128, activation="relu"))
    if dropout_fc != None:
      model.add(Dropout(dropout_fc))
    if batch_normalization != None:
      model.add(BatchNormalization())
  
  model.add(Dense(number_of_classes, activation="softmax"))
  
  model.compile(
      loss="categorical_crossentropy",
      optimizer=optimizer,
      metrics=["accuracy"])
  
  return model  

In [0]:
def train_model(
    conv_num,
    fc_num,
    optimizer,
    epochs,
    pooling_iter_num=1,
    dropout_fc=None,
    dropout_conv=None,
    batch_normalization=None,
    image_augmentation=False,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False,
    vertiacal_flip=False,
    rotation_range=0,
    brightness_range=None,
    zca_whitening=False,
    callbacks=None):
  
  model = create_base_model(
      input_shape=X_train.shape[1:],
      number_of_classes=10,
      conv_num=conv_num,
      fc_num=fc_num,
      optimizer=optimizer,
      dropout_fc=dropout_fc,
      dropout_conv=dropout_conv,
      batch_normalization=batch_normalization,
      pooling_iter_num=pooling_iter_num)
  
  batch_size = 32
  
  if image_augmentation == True:
    datagen = ImageDataGenerator(
      width_shift_range=width_shift_range,
      height_shift_range=height_shift_range,
      shear_range=shear_range,
      zoom_range=zoom_range,
      horizontal_flip=horizontal_flip,
      vertiacal_flip=vertiacal_flip,
      rotation_range=rotation_range,
      brightness_range=brightness_range,
      zca_whitening=zca_whitening)
    datagen.fit(X_train)
    
    return (model, model.fit_generator(
        datagen.flow(X_train, y_train, batch_size=batch_size),
        steps_per_epoch=(len(X_train) // batch_size),
        epochs=epochs,
        validation_data=(X_val, y_val),
        callbacks=callbacks)
    )
  else:
    return (model, model.fit(
      X_train,
      y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(X_val, y_val),
      shuffle=True,
      callbacks=callbacks))

In [0]:
def test(
    file_name,
    conv_num,
    fc_num,
    epochs,
    pooling_iter_num=1,
    dropout_fc=None,
    dropout_conv=None,
    batch_normalization=False,
    image_augmentation=False,
    width_shift_range=0.0,
    height_shift_range=0.0,
    shear_range=0.0,
    zoom_range=0.0,
    horizontal_flip=False,
    vertiacal_flip=False,
    rotation_range=0,
    brightness_range=None,
    zca_whitening=False,
    callbacks=None):
  
  sgd = keras.optimizers.SGD(lr=0.01,
                           decay=1e-6,
                           momentum=0.9,
                           nesterov=True)
  
  model, hist = train_model(
      conv_num,
      fc_num,
      sgd,
      epochs,
      pooling_iter_num,
      dropout_fc,
      dropout_conv,
      batch_normalization,
      image_augmentation,
      width_shift_range,
      height_shift_range,
      shear_range,
      zoom_range,
      horizontal_flip,
      vertiacal_flip,
      rotation_range,
      brightness_range,
      zca_whitening,
      callbacks)
  
  if "lr" in hist.history:
    hist.history["lr"] = [float(num) for num in hist.history["lr"]]
  
  scores = model.evaluate(X_test, y_test, verbose=1)
  
  print("loss: {0}, accuracy: {1}".format(scores[0], scores[1]))
  
  predicted = model.predict(X_test).tolist()
  
  params = {"conv_num": conv_num,
           "fc_num": fc_num,
           "epochs": epochs,
           "dropout_fc": dropout_fc,
           "dropout_conv": dropout_conv,
           "batch_normalization": batch_normalization,
           "image_augmentation": image_augmentation,
           "width_shift_range": width_shift_range,
           "height_shift_range": height_shift_range,
           "shear_range": shear_range,
           "zoom_range": zoom_range,
           "horizontal_flip": horizontal_flip,
           "rotation_range": rotation_range,
           "brightness_range": brightness_range,
           "zca_whitening": zca_whitening,
           "hist": hist.history,
           "conf": hist.model.get_config(),
           "test_eval_scores": scores,
           "predicted": predicted}
  
  save_file(file_name, params)
  
  return (model, hist, scores)

In [0]:
(m1, h1, s1) = test(file_name="model1.json",
        conv_num=3,
        fc_num=2,
        epochs=5)

In [0]:
(m2, h2, s2) = test(file_name="model2.json",
        conv_num=10,
        fc_num=2,
        epochs=5)

In [0]:
(m3, h3, s3) = test(file_name="model3.json",
        conv_num=30,
        fc_num=2,
        epochs=5)

In [0]:
(m4, h4, s4) = test(file_name="model4.json",
        conv_num=3,
        pooling_iter_num=4,
        fc_num=2,
        epochs=5)

In [0]:
(m5, h5, s5) = test(file_name="model5.json",
        conv_num=8,
        pooling_iter_num=4,
        fc_num=2,
        epochs=5)

In [0]:
(m6, h6, s6) = test(file_name="model6.json",
        conv_num=3,
        pooling_iter_num=4,
        fc_num=2,
        epochs=40)

In [0]:
print("ss")

ss


In [0]:
m6.save("m6.h5")

In [0]:
import pickle
pickle.dump(m6, open("/content/gdrive/My Drive/model6", "wb"))

In [0]:
(m7, h7, s7) = test(file_name="model7.json",
        conv_num=3,
        pooling_iter_num=4,
        fc_num=2,
        epochs=50,
        batch_normalization=True,
        dropout_fc=0.2,
        dropout_conv=0.2)

In [0]:
import pickle
pickle.dump(m7, open("/content/gdrive/My Drive/model7_pickl", "wb"))

# Do tego miejsca nie było ustawiane zmienne `learning rate`

In [0]:
from keras.callbacks import LearningRateScheduler
def get_learning_rate(epoch):
  if epoch < 30:
    return 0.01
  elif epoch < 60:
    return 0.0005
  else:
    return 0.0003

In [0]:
(m8, h8, s8) = test(file_name="model8.json",
        conv_num=3,
        pooling_iter_num=4,
        fc_num=2,
        epochs=80,
        batch_normalization=True,
        dropout_fc=0.2,
        dropout_conv=0.2,
        callbacks=[LearningRateScheduler(get_learning_rate)])

Train on 45000 samples, validate on 5000 samples
Epoch 1/80
45000/45000 [==============================] - 151s 3ms/step - loss: 2.0618 - acc: 0.2389 - val_loss: 2.0600 - val_acc: 0.2200
Epoch 2/80
45000/45000 [==============================] - 133s 3ms/step - loss: 1.7447 - acc: 0.3551 - val_loss: 2.1345 - val_acc: 0.2438
Epoch 3/80
45000/45000 [==============================] - 133s 3ms/step - loss: 1.6148 - acc: 0.4108 - val_loss: 2.1228 - val_acc: 0.2178
Epoch 4/80
45000/45000 [==============================] - 133s 3ms/step - loss: 1.5097 - acc: 0.4498 - val_loss: 1.9717 - val_acc: 0.2812
Epoch 5/80
45000/45000 [==============================] - 132s 3ms/step - loss: 1.4243 - acc: 0.4867 - val_loss: 1.7525 - val_acc: 0.3670
Epoch 6/80
45000/45000 [==============================] - 132s 3ms/step - loss: 1.3385 - acc: 0.5172 - val_loss: 1.7199 - val_acc: 0.3868
Epoch 7/80
45000/45000 [==============================] - 132s 3ms/step - loss: 1.2560 - acc: 0.5512 - val_loss: 1.6412 - v

In [0]:
h8.history

{'acc': [0.23262222222222223],
 'loss': [2.094926867421468],
 'lr': [0.01],
 'val_acc': [0.133],
 'val_loss': [2.599663773727417]}

In [0]:
type(h8.history["lr"][0])

numpy.float32

In [0]:
import numpy
type([numpy.float64(x) for x in h8.history["lr"]][0])
json.dumps([numpy.float64(x) for x in h8.history["lr"]])


In [0]:
pickle.dump(m8, open("/content/gdrive/My Drive/model8_pickl", "wb"))


In [0]:
def read_pickle_model(file_name):
  with open("/content/gdrive/My Drive/{0}".format(file_name)) as f:
    return pickle.load(f)
        

# Próba douczenia modelu przy pomocy `data augmentation`

In [0]:
h7.history

In [0]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True)
datagen.fit(X_train)

In [0]:
batch_size=32
hist7_augm = m7.fit_generator(
  datagen.flow(X_train, y_train, batch_size=batch_size),
  steps_per_epoch=(X_train.shape[0] // batch_size),
  callbacks=[LearningRateScheduler(get_learning_rate)],
  epochs=100,
  validation_data=(X_val, y_val))

hist7_augm.history["lr"] = [float(num) for num in hist7_augm.history["lr"]]

scores = m7.evaluate(X_test, y_test, verbose=1)
  
print("loss: {0}, accuracy: {1}".format(scores[0], scores[1]))
 
predicted = m7.predict(X_test).tolist()

params = {"test_eval_scores": scores,
          "predicted": predicted,
          "hist": hist7_augm.history,
          "config": hist7_augm.model.get_config(),
          "predicted": predicted,
          "epochs": 100,
         "rotation_range": 30,
         "width_shift_range":0.1,
         "height_shift_range":0.1,
         "horizontal_flip":True,
         "vertical_flip":True,
         "lr_scheduler": True}

save_file("model7_with_img_augm.json", params)
pickle.dump(m7, open("/content/gdrive/My Drive/model7_augm_pickl", "wb"))

Epoch 1/100
1406/1406 [==============================] - 138s 98ms/step - loss: 1.1755 - acc: 0.5913 - val_loss: 0.6852 - val_acc: 0.7710
Epoch 2/100
1406/1406 [==============================] - 137s 98ms/step - loss: 1.1148 - acc: 0.6168 - val_loss: 0.7416 - val_acc: 0.7486
Epoch 3/100
1406/1406 [==============================] - 137s 98ms/step - loss: 1.0796 - acc: 0.6276 - val_loss: 0.7057 - val_acc: 0.7664
Epoch 4/100
1406/1406 [==============================] - 137s 98ms/step - loss: 1.0543 - acc: 0.6397 - val_loss: 0.6937 - val_acc: 0.7640
Epoch 5/100
1406/1406 [==============================] - 137s 98ms/step - loss: 1.0289 - acc: 0.6462 - val_loss: 0.7083 - val_acc: 0.7618
Epoch 6/100
1406/1406 [==============================] - 138s 98ms/step - loss: 1.0110 - acc: 0.6544 - val_loss: 0.7548 - val_acc: 0.7538
Epoch 7/100
1406/1406 [==============================] - 137s 98ms/step - loss: 0.9938 - acc: 0.6597 - val_loss: 0.7166 - val_acc: 0.7656
Epoch 8/100
1406/1406 [===========